# Autores Waren Sanchez | 2023-1198

# PARTE 1 DEL PROYECTO:

 A continuación se trabajaran los puntos de la sección "Detalles de la Implementación" del documento PDF sobre la Descripción del Proyecto.

# 0-) Importaciones:

0.1-) Importación de librerías:

In [107]:
# Para manejo y manipulación de datos
import pandas as pd
import numpy as np

# Para visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns

# Para preprocesamiento de datos
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

# Para modelos de Machine Learning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

# Para evaluación de modelos
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

0.2-) Importación del dataset crudo:

In [108]:
# Rutas:

ruta_dekho = './Datasets_CSV/CAR_DETAILS_FROM_CAR_DEKHO.csv'
ruta_prediction = './Datasets_CSV/car_price_prediction.csv'
ruta_used = './Datasets_CSV/Used_Car_Dataset.csv'
ruta_dvm = './Datasets_CSV/DVM-CAR_Dataset-Ad_table.csv'
ruta_world = './Datasets_CSV/CarsDataWorld.csv'


In [109]:
# Datasets:

df_dekho = pd.read_csv(ruta_dekho)
df_prediction = pd.read_csv(ruta_prediction)
df_used = pd.read_csv(ruta_used)
df_dvm = pd.read_csv(ruta_dvm, low_memory=False)
df_world = pd.read_csv(ruta_world)


1.1-) Antes de todo el proceso de arreglar los datos y combinarlos en uno solo, primero debemos visualizar cada archivo, nos interesa ver cuales columnas coinciden en cuanto a la información que ofrecen y su tipo de dato. Asi sabremos a cuales deberías actualizar su tipo de dato, a cuales debemos fusionar en una sola basándonos en su coincidencia y a cuales deberíamos eliminar debido a que no logro coincidir entre los 5 datasets.

Solo se tomaran en cuenta los datos que coincidan en cuanto a información entre los 5 datasets, los que no logren coincidir con los demás serán eliminados.

In [110]:
# Lista de nombres de archivos y etiquetas
files = [
    ("CAR_DETAILS_FROM_CAR_DEKHO.csv", "Car Dekho"),
    ("car_price_prediction.csv", "Car Price Prediction"),
    ("DVM-CAR_Dataset-Ad_table.csv", "DVM Car Dataset"),
    ("Used_Car_Auction_Prices_car_prices.csv", "Used Car Auction Prices"),
    ("Used_Car_Dataset.csv", "Used Car Dataset")
]

# Inspección de columnas y tipos de datos
for file, label in files:
    print(f"--- {label}: {file} ---")
    try:
        # Lee solo las primeras filas para inspección
        df = pd.read_csv(f'./Datasets_CSV/{file}', nrows=10)
        print("Columnas y tipos de datos:")
        print(df.dtypes)
        print("\n")
    except Exception as e:
        print(f"Error al leer el archivo {file}: {e}\n")

--- Car Dekho: CAR_DETAILS_FROM_CAR_DEKHO.csv ---
Columnas y tipos de datos:
name             object
year              int64
selling_price     int64
km_driven         int64
fuel             object
seller_type      object
transmission     object
owner            object
dtype: object


--- Car Price Prediction: car_price_prediction.csv ---
Columnas y tipos de datos:
ID                    int64
Price                 int64
Levy                 object
Manufacturer         object
Model                object
Prod. year            int64
Category             object
Leather interior     object
Fuel type            object
Engine volume       float64
Mileage              object
Cylinders           float64
Gear box type        object
Drive wheels         object
Doors                object
Wheel                object
Color                object
Airbags               int64
dtype: object


--- DVM Car Dataset: DVM-CAR_Dataset-Ad_table.csv ---
Columnas y tipos de datos:
Maker           object
Genmodel 

# Crearemos el nuevo csv que se encargara de recibir los datos de las demas columnas, es decir, sera el destino de fusion para los demas datasets.

In [111]:
# Crear un DataFrame con columnas específicas y tipos de datos predefinidos
data = {
    'Name': [],  # Cadenas de texto
    'Year': [],  # Números enteros
    'Price': [],  # Números con decimales
    'Mileage': [],  # Números con decimales
    'Fuel': [],  # Cadenas de texto
    'Transmission': []  # Cadenas de texto
}

# Crear el DataFrame
df_fusion = pd.DataFrame(data)

# Cambiar los tipos de datos según sea necesario (aunque pandas lo hará automáticamente en muchos casos)
df_fusion['Name'] = df_fusion['Name'].astype(str)
df_fusion['Year'] = df_fusion['Year'].astype(int)
df_fusion['Price'] = df_fusion['Price'].astype(float)
df_fusion['Mileage'] = df_fusion['Mileage'].astype(float)
df_fusion['Fuel'] = df_fusion['Fuel'].astype(str)
df_fusion['Transmission'] = df_fusion['Transmission'].astype(str)

# Guardar el DataFrame en un nuevo archivo CSV
ruta_fusion = './Datasets_CSV/dataset_fusionado.csv'

df_fusion.to_csv(ruta_fusion, index=False)

# Verificar el resultado
print(f"El archivo CSV 'dataset_fusionado.csv' ha sido creado con las siguientes columnas y tipos de datos:")
print(df_fusion.dtypes)

df_fusion = pd.read_csv(ruta_fusion)

El archivo CSV 'dataset_fusionado.csv' ha sido creado con las siguientes columnas y tipos de datos:
Name             object
Year              int64
Price           float64
Mileage         float64
Fuel             object
Transmission     object
dtype: object


Para abordar la eliminación de columnas que no son comunes en los cinco datasets, primero analizaremos las columnas presentes en cada uno y luego determinaremos cuáles son las que comparten información en común. Una vez identificadas, se mantendrán únicamente las columnas comunes, eliminando las restantes.

# Empezaremos el proceso de limpieza y organizacion adecuada de los datasets.

In [112]:
# Asignamos un nuevo dataset a df_modificar
df_Modificar = df_dekho

Aca eliminaremos las columnas de df_dekho que no son relevantes.

Este es df_dekho antes de eliminar las columnas no relevantes:

In [113]:
print(df_dekho.dtypes)

name             object
year              int64
selling_price     int64
km_driven         int64
fuel             object
seller_type      object
transmission     object
owner            object
dtype: object


Eliminaremos las columnas innecesarias del dataset.

In [114]:
# display
print("Original 'df_dekho' CSV Data: \n")
print(df_Modificar.dtypes)

# Empezaremos a eliminar lo indeseado.
columnasRelevantes = ['name', 'year', 'selling_price', 'km_driven', 'fuel', 'transmission']
columnasParaEliminar = []

# Identificamos y guardamos las columnas irrelevantes.
for col in df_Modificar.columns:
    if col not in columnasRelevantes:
        columnasParaEliminar.append(col)

# Eliminamos las columnas irrelevantes que fueron guardaas.
df_Modificar = df_Modificar.drop(columnasParaEliminar, axis=1)

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_dekho, index=False)

# display
print("\nCSV Data after deleting the columns:\n")
print(df_Modificar.dtypes)

Original 'df_dekho' CSV Data: 

name             object
year              int64
selling_price     int64
km_driven         int64
fuel             object
seller_type      object
transmission     object
owner            object
dtype: object

CSV Data after deleting the columns:

name             object
year              int64
selling_price     int64
km_driven         int64
fuel             object
transmission     object
dtype: object


Bien, ahora seguiremos con borrar las filas con datos nulos o NaN. Primero veamos cuantas filas tiene df_dekho.

In [115]:
# Mostrar el DataFrame original
print("DataFrame Original:")
print(df_Modificar)

# Paso 1: Reemplazar cadenas vacías y None por NaN
df_Modificar.replace(['', None], pd.NA, inplace=True)

# Paso 2: Eliminar filas con cualquier valor NaN
df_Modificar = df_Modificar.dropna()

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_dekho, index=False)

# Mostrar el DataFrame después de eliminar las filas con NaN, '', o None
print("\nDataFrame después de eliminar filas con valores NaN, vacíos y None:")
print(df_Modificar)

# Verificar la cantidad de filas después de la limpieza
print(f"\nCantidad de filas después de la limpieza: {df_Modificar.shape[0]}")

DataFrame Original:
                                     name  year  selling_price  km_driven  \
0                           Maruti 800 AC  2007          60000      70000   
1                Maruti Wagon R LXI Minor  2007         135000      50000   
2                    Hyundai Verna 1.6 SX  2012         600000     100000   
3                  Datsun RediGO T Option  2017         250000      46000   
4                   Honda Amaze VX i-DTEC  2014         450000     141000   
...                                   ...   ...            ...        ...   
4335  Hyundai i20 Magna 1.4 CRDi (Diesel)  2014         409999      80000   
4336           Hyundai i20 Magna 1.4 CRDi  2014         409999      80000   
4337                  Maruti 800 AC BSIII  2009         110000      83000   
4338     Hyundai Creta 1.6 CRDi SX Option  2016         865000      90000   
4339                     Renault KWID RXT  2016         225000      40000   

        fuel transmission  
0     Petrol       Manual  

Segun parece, dekho no posee valores nulos.

Ahora convirtamos los tipos de datos del dataset para que sean adecuados para la fusion.

In [116]:
print("Tipos de datos de df_dekho antes de la conversion:\n")
print(df_Modificar.dtypes)

# Cambiar los tipos de datos de las columnas
df_Modificar['selling_price'] = df_Modificar['selling_price'].astype(float)
df_Modificar['km_driven'] = df_Modificar['km_driven'].astype(float)

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_dekho, index=False)

print("\nTipos de datos de df_dekho despues de la conversion:\n")
print(df_Modificar.dtypes)

Tipos de datos de df_dekho antes de la conversion:

name             object
year              int64
selling_price     int64
km_driven         int64
fuel             object
transmission     object
dtype: object

Tipos de datos de df_dekho despues de la conversion:

name              object
year               int64
selling_price    float64
km_driven        float64
fuel              object
transmission      object
dtype: object


Ahora funcionaremos sus datos al dataset_fusionado.

In [117]:
print(f"Cantidad de columnas del CSV antes de la Fusion: {df_fusion.shape[0]}")

# Seleccionar solo las columnas necesarias de df_Modificar
df_Modificar_select = df_Modificar[['name', 'year', 'selling_price', 'km_driven', 'fuel', 'transmission']]

# Renombrar las columnas para que coincidan con las de df_fusion si es necesario
df_Modificar_select.columns = ['Name', 'Year', 'Price', 'Mileage', 'Fuel', 'Transmission']

# Concatenar df_Modificar_select a df_fusion
df_fusion = pd.concat([df_fusion, df_Modificar_select], ignore_index=True)

# Guardamos los cambios en el archivo CSV
df_fusion.to_csv(ruta_fusion, index=False)

# Verificar el resultado de la fusión
print(f"Cantidad de filas del CSV después de la fusión: {df_fusion.shape[0]}")

Cantidad de columnas del CSV antes de la Fusion: 0
Cantidad de filas del CSV después de la fusión: 4340


C:\Users\warre\AppData\Local\Temp\ipykernel_8184\3117655699.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_fusion = pd.concat([df_fusion, df_Modificar_select], ignore_index=True)


# Ahora continuamos con df_prediction

In [118]:
# Asignamos un nuevo dataset para df_Modificar.
df_Modificar = df_prediction

Ahora eliminaremos las columnas que no sean relevantes de df_prediction

In [119]:

# Concatenaremos las columnas Model y Manufacturer.
df_Modificar['car_name'] = df_Modificar['Manufacturer'] + ' ' + df_Modificar['Model']

# display
print("Original 'df_dekho' CSV Data: \n")
print(df_Modificar.dtypes)

# Empezaremos a eliminar lo indeseado.
columnasRelevantes = ['car_name', 'Prod. year', 'Price', 'Mileage', 'Fuel type', 'Gear box type']
columnasParaEliminar = []

# Identificamos y guardamos las columnas irrelevantes.
for col in df_Modificar.columns:
    if col not in columnasRelevantes:
        columnasParaEliminar.append(col)

# Eliminamos las columnas irrelevantes que fueron guardaas.
df_Modificar = df_Modificar.drop(columnasParaEliminar, axis=1)

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_prediction, index=False)

# display
print("\nCSV Data after deleting the columns:\n")
print(df_Modificar.dtypes)

Original 'df_dekho' CSV Data: 

ID                    int64
Price                 int64
Levy                 object
Manufacturer         object
Model                object
Prod. year            int64
Category             object
Leather interior     object
Fuel type            object
Engine volume        object
Mileage              object
Cylinders           float64
Gear box type        object
Drive wheels         object
Doors                object
Wheel                object
Color                object
Airbags               int64
car_name             object
dtype: object

CSV Data after deleting the columns:

Price             int64
Prod. year        int64
Fuel type        object
Mileage          object
Gear box type    object
car_name         object
dtype: object


Primero voy a eliminar cualquier valor nulo que tenga el dataset.

In [120]:
# Mostrar el DataFrame original
print("DataFrame Original:")
print(df_Modificar)

# Paso 1: Reemplazar cadenas vacías y None por NaN
df_Modificar.replace(['', None], pd.NA, inplace=True)

# Paso 2: Eliminar filas con cualquier valor NaN
df_Modificar = df_Modificar.dropna()

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_prediction, index=False)

# Mostrar el DataFrame después de eliminar las filas con NaN, '', o None
print("\nDataFrame después de eliminar filas con valores NaN, vacíos y None:")
print(df_Modificar)

# Verificar la cantidad de filas después de la limpieza
print(f"\nCantidad de filas después de la limpieza: {df_Modificar.shape[0]}")

DataFrame Original:
       Price  Prod. year Fuel type    Mileage Gear box type  \
0      13328        2010    Hybrid  186005 km     Automatic   
1      16621        2011    Petrol  192000 km     Tiptronic   
2       8467        2006    Petrol  200000 km      Variator   
3       3607        2011    Hybrid  168966 km     Automatic   
4      11726        2014    Petrol   91901 km     Automatic   
...      ...         ...       ...        ...           ...   
19232   8467        1999       CNG  300000 km        Manual   
19233  15681        2011    Petrol  161600 km     Tiptronic   
19234  26108        2010    Diesel  116365 km     Automatic   
19235   5331        2007    Diesel   51258 km     Automatic   
19236    470        2012    Hybrid  186923 km     Automatic   

                    car_name  
0               LEXUS RX 450  
1          CHEVROLET Equinox  
2                  HONDA FIT  
3                FORD Escape  
4                  HONDA FIT  
...                      ...  
19232 

Tal parece que este tampoco tiene valores nulos, lo cual es perfecto, asi no perdemos datos.

Eliminemos km de Mileage, ya que nos impedira convertirlo a float durante la conversion, vamos a corregir eso.

In [121]:
# Mileage antes de quitarle km:
print("Mileage antes de quitarle km:\n")
print(df_Modificar['Mileage'])

# Limpiar la columna de Mileage de km
df_Modificar['Mileage'] = df_Modificar['Mileage'].replace({' km': '', ',': ''}, regex=True)

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_prediction, index=False)

# Mileage despues de quitarle km:
print("Mileage despues de quitarle km:\n")
print(df_Modificar['Mileage'])

Mileage antes de quitarle km:

0        186005 km
1        192000 km
2        200000 km
3        168966 km
4         91901 km
           ...    
19232    300000 km
19233    161600 km
19234    116365 km
19235     51258 km
19236    186923 km
Name: Mileage, Length: 19237, dtype: object
Mileage despues de quitarle km:

0        186005
1        192000
2        200000
3        168966
4         91901
          ...  
19232    300000
19233    161600
19234    116365
19235     51258
19236    186923
Name: Mileage, Length: 19237, dtype: object


Ahora convirtamos los tipos de datos del dataset para que sean adecuados para la fusion.

In [122]:
print("Tipos de datos de CSV antes de la conversion:\n")
print(df_Modificar.dtypes)

# Cambiar los tipos de datos de las columnas
df_Modificar['Price'] = df_Modificar['Price'].astype(float)
df_Modificar['Mileage'] = df_Modificar['Mileage'].astype(float)

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_prediction, index=False)

print("\nTipos de datos del CSV despues de la conversion:\n")
print(df_Modificar.dtypes)

Tipos de datos de CSV antes de la conversion:

Price             int64
Prod. year        int64
Fuel type        object
Mileage          object
Gear box type    object
car_name         object
dtype: object

Tipos de datos del CSV despues de la conversion:

Price            float64
Prod. year         int64
Fuel type         object
Mileage          float64
Gear box type     object
car_name          object
dtype: object


Ahora funcionaremos sus datos al dataset_fusionado.

In [123]:
print(f"Cantidad de columnas del CSV antes de la Fusion: {df_fusion.shape[0]}")

# Seleccionar solo las columnas necesarias de df_Modificar
df_Modificar_select = df_Modificar[['car_name', 'Prod. year', 'Price', 'Mileage', 'Fuel type', 'Gear box type']]

# Renombrar las columnas para que coincidan con las de df_fusion si es necesario
df_Modificar_select.columns = ['Name', 'Year', 'Price', 'Mileage', 'Fuel', 'Transmission']

# Concatenar df_Modificar_select a df_fusion
df_fusion = pd.concat([df_fusion, df_Modificar_select], ignore_index=True)

# Guardamos los cambios en el archivo CSV
df_fusion.to_csv(ruta_fusion, index=False)

# Verificar el resultado de la fusión
print(f"Cantidad de filas del CSV después de la fusión: {df_fusion.shape[0]}")


Cantidad de columnas del CSV antes de la Fusion: 4340
Cantidad de filas del CSV después de la fusión: 23577


# Ahora continuamos con df_used

In [124]:
# Asignamos un nuevo dataset para df_Modificar.
df_Modificar = df_used

Ahora eliminaremos las columnas que no sean relevantes de df_prediction

In [125]:
# display
print("Original CSV Data: \n")
print(df_Modificar.dtypes)

# Empezaremos a eliminar lo indeseado.
columnasRelevantes = ['car_name', 'manufacturing_year', 'price(in lakhs)', 'kms_driven', 'fuel_type', 'transmission']
columnasParaEliminar = []

# Identificamos y guardamos las columnas irrelevantes.
for col in df_Modificar.columns:
    if col not in columnasRelevantes:
        columnasParaEliminar.append(col)

# Eliminamos las columnas irrelevantes que fueron guardaas.
df_Modificar = df_Modificar.drop(columnasParaEliminar, axis=1)

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_used, index=False)

# display
print("\nCSV Data after deleting the columns:\n")
print(df_Modificar.dtypes)

Original CSV Data: 

Unnamed: 0              int64
car_name               object
registration_year      object
insurance_validity     object
fuel_type              object
seats                   int64
kms_driven              int64
ownsership             object
transmission           object
manufacturing_year     object
mileage(kmpl)         float64
engine(cc)            float64
max_power(bhp)        float64
torque(Nm)            float64
price(in lakhs)       float64
dtype: object

CSV Data after deleting the columns:

car_name               object
fuel_type              object
kms_driven              int64
transmission           object
manufacturing_year     object
price(in lakhs)       float64
dtype: object


Primero voy a eliminar cualquier valor nulo que tenga el dataset.

In [126]:
# Mostrar el DataFrame original
print("DataFrame Original:")
print(df_Modificar)

# Paso 1: Reemplazar cadenas vacías y None por NaN
df_Modificar.replace(['', None], pd.NA, inplace=True)

# Paso 2: Eliminar filas con cualquier valor NaN
df_Modificar = df_Modificar.dropna()

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_used, index=False)

# Mostrar el DataFrame después de eliminar las filas con NaN, '', o None
print("\nDataFrame después de eliminar filas con valores NaN, vacíos y None:")
print(df_Modificar)

# Verificar la cantidad de filas después de la limpieza
print(f"\nCantidad de filas después de la limpieza: {df_Modificar.shape[0]}")

DataFrame Original:
                                               car_name fuel_type  kms_driven  \
0                       2017 Mercedes-Benz S-Class S400    Petrol       56000   
1     2020 Nissan Magnite Turbo CVT XV Premium Opt BSVI    Petrol       30615   
2                          2018 BMW X1 sDrive 20d xLine    Diesel       24000   
3                              2019 Kia Seltos GTX Plus    Petrol       18378   
4                       2019 Skoda Superb LK 1.8 TSI AT    Petrol       44900   
...                                                 ...       ...         ...   
1548           2020 Hyundai Creta SX Opt Diesel AT BSVI    Diesel       35000   
1549                 2022 Renault KWID 1.0 RXL Opt BSVI    Petrol       10000   
1550                           2017 Honda WR-V i-VTEC S    Petrol       49000   
1551           2018 Volkswagen Polo 1.0 MPI Comfortline    Petrol       40000   
1552                        2018 Maruti Swift Dzire VXI    Petrol       34756   

     tr

Tal parece que este tampoco tiene valores nulos, lo cual es perfecto, asi no perdemos datos.

La columna manufacturing_year es de tipo object y sus datos son de años, por lo que se debe convertir a entero. El problema es que hay algunos impostores de tipo string. Por lo que filtraremos esos casos y eliminaremos esas filas.

In [127]:
print("Cantidad de Filas del CSV antes de la limpieza de impostores:\n")
print(df_Modificar.shape[0])

# Paso 1: Filtrar filas donde 'manufacturing_year' no empieza con un número
df_Modificar = df_Modificar[df_Modificar['manufacturing_year'].str.match(r'^\d')]

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_used, index=False)

print("Cantidad de Filas del CSV despues de la limpieza de impostores:\n")
print(df_Modificar.shape[0])

Cantidad de Filas del CSV antes de la limpieza de impostores:

1553
Cantidad de Filas del CSV despues de la limpieza de impostores:

1503


Ahora convirtamos los tipos de datos del dataset para que sean adecuados para la fusion.

In [128]:
print("Tipos de datos de CSV antes de la conversion:\n")
print(df_Modificar.dtypes)

# Cambiar los tipos de datos de las columnas
df_Modificar['manufacturing_year'] = df_Modificar['manufacturing_year'].astype(int)
df_Modificar['kms_driven'] = df_Modificar['kms_driven'].astype(float)

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_used, index=False)

print("\nTipos de datos del CSV despues de la conversion:\n")
print(df_Modificar.dtypes)

Tipos de datos de CSV antes de la conversion:

car_name               object
fuel_type              object
kms_driven              int64
transmission           object
manufacturing_year     object
price(in lakhs)       float64
dtype: object

Tipos de datos del CSV despues de la conversion:

car_name               object
fuel_type              object
kms_driven            float64
transmission           object
manufacturing_year      int64
price(in lakhs)       float64
dtype: object


Ahora funcionaremos sus datos al dataset_fusionado.

In [129]:
print(f"Cantidad de columnas del CSV antes de la Fusion: {df_fusion.shape[0]}")

# Seleccionar solo las columnas necesarias de df_Modificar
df_Modificar_select = df_Modificar[['car_name', 'manufacturing_year', 'price(in lakhs)', 'kms_driven', 'fuel_type', 'transmission']]

# Renombrar las columnas para que coincidan con las de df_fusion si es necesario
df_Modificar_select.columns = ['Name', 'Year', 'Price', 'Mileage', 'Fuel', 'Transmission']

# Concatenar df_Modificar_select a df_fusion
df_fusion = pd.concat([df_fusion, df_Modificar_select], ignore_index=True)

# Guardamos los cambios en el archivo CSV
df_fusion.to_csv(ruta_fusion, index=False)

# Verificar el resultado de la fusión
print(f"Cantidad de filas del CSV después de la fusión: {df_fusion.shape[0]}")


Cantidad de columnas del CSV antes de la Fusion: 23577
Cantidad de filas del CSV después de la fusión: 25080


# Continuamos con df_dvm

In [130]:
# Asignamos un nuevo dataset para df_Modificar.
df_Modificar = df_dvm

Ahora eliminaremos las columnas que no sean relevantes de df_prediction

In [131]:
df_Modificar['car_name'] = df_Modificar['Maker'] + ' ' + df_Modificar['Genmodel'] + ' ' + df_Modificar['Color']

# display
print("Original CSV Data: \n")
print(df_Modificar.dtypes)

# Empezaremos a eliminar lo indeseado.
columnasRelevantes = ['car_name', 'Adv_year', 'Price', 'Runned_Miles', 'Fuel_type', 'Gearbox']
columnasParaEliminar = []

# Identificamos y guardamos las columnas irrelevantes.
for col in df_Modificar.columns:
    if col not in columnasRelevantes:
        columnasParaEliminar.append(col)

# Eliminamos las columnas irrelevantes que fueron guardaas.
df_Modificar = df_Modificar.drop(columnasParaEliminar, axis=1)

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_dvm, index=False)

# display
print("\nCSV Data after deleting the columns:\n")
print(df_Modificar.dtypes)

Original CSV Data: 

Maker            object
Genmodel         object
Genmodel_ID      object
Adv_ID           object
Adv_year          int64
Adv_month         int64
Color            object
Reg_year        float64
Bodytype         object
Runned_Miles     object
Engin_size       object
Gearbox          object
Fuel_type        object
Price            object
Seat_num        float64
Door_num        float64
car_name         object
dtype: object

CSV Data after deleting the columns:

Adv_year         int64
Runned_Miles    object
Gearbox         object
Fuel_type       object
Price           object
car_name        object
dtype: object


Primero voy a eliminar cualquier valor nulo que tenga el dataset.

In [132]:
# Mostrar el DataFrame original
print("DataFrame Original:")
print(df_Modificar)

# Paso 1: Reemplazar cadenas vacías y None por NaN
df_Modificar.replace(['', None], pd.NA, inplace=True)

# Paso 2: Eliminar filas con cualquier valor NaN
df_Modificar = df_Modificar.dropna()

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_dvm, index=False)

# Mostrar el DataFrame después de eliminar las filas con NaN, '', o None
print("\nDataFrame después de eliminar filas con valores NaN, vacíos y None:")
print(df_Modificar)

# Verificar la cantidad de filas después de la limpieza
print(f"\nCantidad de filas después de la limpieza: {df_Modificar.shape[0]}")

DataFrame Original:
        Adv_year Runned_Miles    Gearbox Fuel_type  Price  \
0           2018        60000  Automatic    Petrol  21500   
1           2018        44000  Automatic    Petrol  28750   
2           2017        55000  Automatic    Petrol  29999   
3           2018        14000  Automatic    Petrol  34948   
4           2017        61652  Automatic    Petrol  26555   
...          ...          ...        ...       ...    ...   
268250      2018         1800     Manual    Petrol   8750   
268251      2018         2009     Manual       NaN   7995   
268252      2018            6     Manual    Petrol  27950   
268253      2018         1538     Manual    Petrol  34950   
268254      2018          500     Manual    Petrol  29995   

                      car_name  
0        Bentley Arnage Silver  
1          Bentley Arnage Grey  
2          Bentley Arnage Blue  
3         Bentley Arnage Green  
4          Bentley Arnage Grey  
...                        ...  
268250  Westfiel

Tal parece que este tampoco tiene valores nulos, lo cual es perfecto, asi no perdemos datos.

Las columna Runned_Miles y Price tienen el problema de que hay algunos impostores de tipo string entre ellas. Por lo que filtraremos esos casos y eliminaremos esas filas.

In [133]:
print("Cantidad de Filas del CSV antes de la limpieza de impostores:\n")
print(df_Modificar.shape[0])

# Paso 1: Filtrar filas seleccionadas donde no empieza con un número
df_Modificar = df_Modificar[df_Modificar['Price'].str.match(r'^\d.*\d$')]
df_Modificar = df_Modificar[df_Modificar['Runned_Miles'].str.match(r'^\d.*\d$')]
# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_dvm, index=False)

print("Cantidad de Filas del CSV despues de la limpieza de impostores:\n")
print(df_Modificar.shape[0])

Cantidad de Filas del CSV antes de la limpieza de impostores:

245042
Cantidad de Filas del CSV despues de la limpieza de impostores:

242361


Ahora convirtamos los tipos de datos del dataset para que sean adecuados para la fusion.

In [134]:
print(df_Modificar.dtypes)

Adv_year         int64
Runned_Miles    object
Gearbox         object
Fuel_type       object
Price           object
car_name        object
dtype: object


In [135]:
print("Tipos de datos de CSV antes de la conversion:\n")
print(df_Modificar.dtypes)

# Cambiar los tipos de datos de las columnas
df_Modificar['Price'] = df_Modificar['Price'].astype(float)
df_Modificar['Runned_Miles'] = df_Modificar['Runned_Miles'].astype(float)

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_dvm, index=False)

print("\nTipos de datos del CSV despues de la conversion:\n")
print(df_Modificar.dtypes)

Tipos de datos de CSV antes de la conversion:

Adv_year         int64
Runned_Miles    object
Gearbox         object
Fuel_type       object
Price           object
car_name        object
dtype: object

Tipos de datos del CSV despues de la conversion:

Adv_year          int64
Runned_Miles    float64
Gearbox          object
Fuel_type        object
Price           float64
car_name         object
dtype: object


Ahora funcionaremos sus datos al dataset_fusionado.

In [136]:
print(f"Cantidad de columnas del CSV antes de la Fusion: {df_fusion.shape[0]}")

# Seleccionar solo las columnas necesarias de df_Modificar
df_Modificar_select = df_Modificar[['car_name', 'Adv_year', 'Price', 'Runned_Miles', 'Fuel_type', 'Gearbox']]

# Renombrar las columnas para que coincidan con las de df_fusion si es necesario
df_Modificar_select.columns = ['Name', 'Year', 'Price', 'Mileage', 'Fuel', 'Transmission']

# Concatenar df_Modificar_select a df_fusion
df_fusion = pd.concat([df_fusion, df_Modificar_select], ignore_index=True)

# Guardamos los cambios en el archivo CSV
df_fusion.to_csv(ruta_fusion, index=False)

# Verificar el resultado de la fusión
print(f"Cantidad de filas del CSV después de la fusión: {df_fusion.shape[0]}")


Cantidad de columnas del CSV antes de la Fusion: 25080
Cantidad de filas del CSV después de la fusión: 267441


# Continuamos con df_world

In [137]:
# Asignamos un nuevo dataset para df_Modificar.
df_Modificar = df_world

In [138]:
print(df_world)

               model  year  price transmission  mileage fuelType  tax   mpg  \
0                I10  2017   7495       Manual    11630   Petrol  145  60.1   
1               Polo  2017  10989       Manual     9200   Petrol  145  58.9   
2           2 Series  2019  27990    Semi-Auto     1614   Diesel  145  49.6   
3       Yeti Outdoor  2017  12495       Manual    30960   Diesel  150  62.8   
4             Fiesta  2017   7999       Manual    19353   Petrol  125  54.3   
...              ...   ...    ...          ...      ...      ...  ...   ...   
97707         Fiesta  2017  10447    Automatic     8337   Petrol  145  54.3   
97708       3 Series  2014  14995       Manual    25372   Diesel   30  61.4   
97709         Fiesta  2017   8950       Manual    19910   Petrol  125  54.3   
97710          Astra  2017  10700    Automatic    24468   Petrol  125  50.4   
97711    Grandland X  2019  15798       Manual    10586   Diesel  150  48.7   

       engineSize Manufacturer  
0             1.0 

Ahora eliminaremos las columnas que no sean relevantes de df_prediction

In [139]:
df_Modificar['car_name'] = df_Modificar['Manufacturer'] + ' ' + df_Modificar['model']

# display
print("Original CSV Data: \n")
print(df_Modificar.dtypes)

# Empezaremos a eliminar lo indeseado.
columnasRelevantes = ['car_name', 'year', 'price', 'mileage', 'fuelType', 'transmission']
columnasParaEliminar = []

# Identificamos y guardamos las columnas irrelevantes.
for col in df_Modificar.columns:
    if col not in columnasRelevantes:
        columnasParaEliminar.append(col)

# Eliminamos las columnas irrelevantes que fueron guardaas.
df_Modificar = df_Modificar.drop(columnasParaEliminar, axis=1)

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_world, index=False)

# display
print("\nCSV Data after deleting the columns:\n")
print(df_Modificar.dtypes)

Original CSV Data: 

model            object
year              int64
price             int64
transmission     object
mileage           int64
fuelType         object
tax               int64
mpg             float64
engineSize      float64
Manufacturer     object
car_name         object
dtype: object

CSV Data after deleting the columns:

year             int64
price            int64
transmission    object
mileage          int64
fuelType        object
car_name        object
dtype: object


Primero voy a eliminar cualquier valor nulo que tenga el dataset.

In [140]:
# Mostrar el DataFrame original
print("DataFrame Original:")
print(df_Modificar)

# Paso 1: Reemplazar cadenas vacías y None por NaN
df_Modificar.replace(['', None], pd.NA, inplace=True)

# Paso 2: Eliminar filas con cualquier valor NaN
df_Modificar = df_Modificar.dropna()

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_world, index=False)

# Mostrar el DataFrame después de eliminar las filas con NaN, '', o None
print("\nDataFrame después de eliminar filas con valores NaN, vacíos y None:")
print(df_Modificar)

# Verificar la cantidad de filas después de la limpieza
print(f"\nCantidad de filas después de la limpieza: {df_Modificar.shape[0]}")

DataFrame Original:
       year  price transmission  mileage fuelType               car_name
0      2017   7495       Manual    11630   Petrol            hyundi  I10
1      2017  10989       Manual     9200   Petrol       volkswagen  Polo
2      2019  27990    Semi-Auto     1614   Diesel          BMW  2 Series
3      2017  12495       Manual    30960   Diesel    skoda  Yeti Outdoor
4      2017   7999       Manual    19353   Petrol           ford  Fiesta
...     ...    ...          ...      ...      ...                    ...
97707  2017  10447    Automatic     8337   Petrol           ford  Fiesta
97708  2014  14995       Manual    25372   Diesel          BMW  3 Series
97709  2017   8950       Manual    19910   Petrol           ford  Fiesta
97710  2017  10700    Automatic    24468   Petrol        vauxhall  Astra
97711  2019  15798       Manual    10586   Diesel  vauxhall  Grandland X

[97712 rows x 6 columns]

DataFrame después de eliminar filas con valores NaN, vacíos y None:
       ye

Tal parece que este tampoco tiene valores nulos, lo cual es perfecto, asi no perdemos datos.

Ahora convirtamos los tipos de datos del dataset para que sean adecuados para la fusion.

In [141]:
print(df_Modificar.dtypes)

year             int64
price            int64
transmission    object
mileage          int64
fuelType        object
car_name        object
dtype: object


In [142]:
print("Tipos de datos de CSV antes de la conversion:\n")
print(df_Modificar.dtypes)

# Cambiar los tipos de datos de las columnas
df_Modificar['price'] = df_Modificar['price'].astype(float)
df_Modificar['mileage'] = df_Modificar['mileage'].astype(float)

# Guardamos los cambios del CSV en el mismo archivo CSV.
df_Modificar.to_csv(ruta_world, index=False)

print("\nTipos de datos del CSV despues de la conversion:\n")
print(df_Modificar.dtypes)

Tipos de datos de CSV antes de la conversion:

year             int64
price            int64
transmission    object
mileage          int64
fuelType        object
car_name        object
dtype: object

Tipos de datos del CSV despues de la conversion:

year              int64
price           float64
transmission     object
mileage         float64
fuelType         object
car_name         object
dtype: object


Ahora funcionaremos sus datos al dataset_fusionado.

In [143]:
print(f"Cantidad de columnas del CSV antes de la Fusion: {df_fusion.shape[0]}")

# Seleccionar solo las columnas necesarias de df_Modificar
df_Modificar_select = df_Modificar[['car_name', 'year', 'price', 'mileage', 'fuelType', 'transmission']]

# Renombrar las columnas para que coincidan con las de df_fusion si es necesario
df_Modificar_select.columns = ['Name', 'Year', 'Price', 'Mileage', 'Fuel', 'Transmission']

# Concatenar df_Modificar_select a df_fusion
df_fusion = pd.concat([df_fusion, df_Modificar_select], ignore_index=True)

# Guardamos los cambios en el archivo CSV
df_fusion.to_csv(ruta_fusion, index=False)

# Verificar el resultado de la fusión
print(f"Cantidad de filas del CSV después de la fusión: {df_fusion.shape[0]}")


Cantidad de columnas del CSV antes de la Fusion: 267441
Cantidad de filas del CSV después de la fusión: 365153


# Al final el archivo fusionado nos quedo con 365,153 filas y 6 columnas.